In [168]:
import torch 
import torch.nn as nn  
import torch.optim as optim 
import torch.utils.data 
import torch.nn.functional as F 
import torchvision 
from torchvision import transforms 
from PIL import Image 


In [169]:
class CNNNet(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__() 
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(64, 192, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3),
            nn.Conv2d(192, 384, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(384,256, kernel_size=3),
            nn.ReLU(),
            # nn.Conv2d(256,256, kernel_size=3),
            # nn.ReLU(),
            # nn.MaxPool2d(kernel_size=3),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6,6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(9216, 4096), # nn.Linear(256*6*6, 4096)
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096,4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )

    def forward(self,x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

cnnnet = CNNNet()

In [170]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0 
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch 
            # print('inputs = ', inputs)
            # print('input shape = ', inputs.size())
            # print('targets = ', targets)
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss, valid_loss, num_correct / num_examples))



In [171]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False 
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
                    ])

train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [172]:
cnnnet.to(device)
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)
train(cnnnet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=20, device=device)


Epoch: 0, Training Loss: 1.01, Validation Loss: 0.70, accuracy = 0.42
Epoch: 1, Training Loss: 0.66, Validation Loss: 0.68, accuracy = 0.50
Epoch: 2, Training Loss: 0.58, Validation Loss: 0.58, accuracy = 0.63
Epoch: 3, Training Loss: 0.50, Validation Loss: 0.38, accuracy = 0.83
Epoch: 4, Training Loss: 0.48, Validation Loss: 0.38, accuracy = 0.83
Epoch: 5, Training Loss: 0.46, Validation Loss: 0.45, accuracy = 0.76
Epoch: 6, Training Loss: 0.41, Validation Loss: 0.45, accuracy = 0.73
Epoch: 7, Training Loss: 0.37, Validation Loss: 0.37, accuracy = 0.80
Epoch: 8, Training Loss: 0.35, Validation Loss: 0.26, accuracy = 0.88
Epoch: 9, Training Loss: 0.32, Validation Loss: 0.38, accuracy = 0.83
Epoch: 10, Training Loss: 0.26, Validation Loss: 0.48, accuracy = 0.79
Epoch: 11, Training Loss: 0.19, Validation Loss: 0.52, accuracy = 0.77
Epoch: 12, Training Loss: 0.19, Validation Loss: 0.43, accuracy = 0.80
Epoch: 13, Training Loss: 0.13, Validation Loss: 0.69, accuracy = 0.76
Epoch: 14, Train